# LSST Simulation generation
This notebook is a wrapper for StarFast, which generates simulated images with realistic stellar distributions and spectra, psf and filters, and detailed atmospheric differential chromatic refraction (DCR)

In [ ]:
import numpy as np
from python.simulation_utils import simulation_wrapper

Set up the general parameters to use for all of the simulations 

In [ ]:
seed = 8
n_star = 10000
n_quasar = 0
attenuation = 20.
wavelength_step = 10
output_directory = "/Users/sullivan/LSST/simulations/test%1i_constantPSF/" % seed
sim = simulation_wrapper(seed=seed, n_star=n_star, n_quasar=n_quasar, output_directory=output_directory,
                         attenuation=attenuation, wavelength_step=wavelength_step,
                         write_catalog=False, write_fits=False, do_simulate=True)

# g-band simulation

In [7]:
band_name = 'g'

In [ ]:
sim = simulation_wrapper(sim, band_name=band_name, write_catalog=True, write_fits=False)

In [ ]:
sim = simulation_wrapper(sim, band_name=band_name, write_catalog=False, write_fits=True)

# u-band simulation

In [6]:
band_name = 'u'

In [7]:
sim = simulation_wrapper(sim, band_name=band_name, write_catalog=True, write_fits=False)

In [ ]:
sim = simulation_wrapper(sim, band_name=band_name, write_catalog=False, write_fits=True)